### 参考
1. [Optimize](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html)
2. [Constrained Optimization demystified, with implementation in Python.](https://towardsdatascience.com/constrained-optimization-demystified-with-implementation-in-python-235639546fa9)
3. [雅可比矩阵](https://baike.baidu.com/item/%E9%9B%85%E5%8F%AF%E6%AF%94%E7%9F%A9%E9%98%B5/10753754?fromtitle=%E9%9B%85%E5%85%8B%E6%AF%94%E7%9F%A9%E9%98%B5&fromid=5726542&fr=aladdin)
4. [Jacobian矩阵和Hessian矩阵](https://www.cnblogs.com/wangyarui/p/6407604.html)

In [1]:
import numpy as np
import scipy.optimize as optimize
import scipy.integrate as integrate

$\large \displaystyle \min_{x_0,x_1} 100(x_1-x_0^2)^2 + (1-x_0)^2 \\
subject \quad to: \\
x_0+2x_1\leq 1\\
x_0^2 + x_1 \leq 1\\
x_0^2-x_1 \leq 1\\
2x_0+x_1 = 1\\
0 \leq x_0 \leq 1\\
-0.5 \leq x_1 \leq 2.0$

In [20]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H

---
$0\leq x_0 \leq 1\\
-0.5 \leq x_1 \leq 2.0$

In [2]:
from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

$x_0+2x_1 \leq 1\\
2x_0+x_1 = 1 \\
\Longrightarrow \\
\begin{bmatrix}
   -\infty \\
   1
\end{bmatrix} \leq \begin{bmatrix}
   1 & 2 \\
   2 & 1
\end{bmatrix} \begin{bmatrix}
   x_0\\
   x_1
\end{bmatrix} \leq \begin{bmatrix}
   1 \\
   1
\end{bmatrix}$

In [3]:
from scipy.optimize import LinearConstraint
linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

$c(x) = \begin{bmatrix}
   x_0^2+x_1 \\
   x_0^2-x_1
\end{bmatrix} \leq 
\begin{bmatrix}
   1 \\
   1
\end{bmatrix} \\
\Longrightarrow \\
J(x) = \begin{bmatrix}
   2x_0 & 1 \\
   2x_0 & -1
\end{bmatrix} \\
\Longrightarrow \\
H(x, v) = \sum_{i=0}^1 v_i \nabla^2c_i(x)=v_0\begin{bmatrix}
   2 & 0\\
   0 & 0
\end{bmatrix} + v_1 \begin{bmatrix}
   2 & 0 \\
   0 & 0
\end{bmatrix}$

In [4]:
def cons_f(x):
    return [x[0]**2 + x[1], x[0]**2 - x[1]]

def cons_J(x):
    return [[2*x[0], 1], [2*x[0], -1]]

def cons_H(x, v):
    return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])

In [5]:
from scipy.optimize import NonlinearConstraint
nonlinear_constraint = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H)

In [6]:
from scipy.sparse import csc_matrix

def cons_H_sparse(x, v):
    return v[0]*csc_matrix([[2, 0], [0, 0]]) + v[1]*csc_matrix([[2, 0], [0, 0]])

nonlinear_constraint2 = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H_sparse)

In [7]:
from scipy.sparse.linalg import LinearOperator

def cons_H_linear_operator(x, v):
    def matvec(p):

        return np.array([p[0]*2*(v[0]+v[1]), 0])

    return LinearOperator((2, 2), matvec=matvec)

nonlinear_constraint3 = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=cons_H_linear_operator)

如果海森矩阵不好算的话

In [8]:
from scipy.optimize import BFGS
nonlinear_constraint3 = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess=BFGS())

使用有限微分估算

In [9]:
nonlinear_constraint4 = NonlinearConstraint(cons_f, -np.inf, 1, jac=cons_J, hess='2-point')

雅克比矩阵也可以用有限微分估算

In [10]:
nonlinear_constraint5 = NonlinearConstraint(cons_f, -np.inf, 1, jac='2-point', hess=BFGS())

In [21]:
x0 = np.array([0.5, 0])
res = optimize.minimize(rosen, x0, method='trust-constr', jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 8, CG iterations: 7, optimality: 2.99e-09, constraint violation: 0.00e+00, execution time: 0.065 s.


In [22]:
res.x

array([0.41494531, 0.17010937])

In [24]:
from scipy.optimize import SR1
res = optimize.minimize(rosen, x0, method='trust-constr',  jac="2-point", hess=SR1(),
               constraints=[linear_constraint, nonlinear_constraint],
               options={'verbose': 1}, bounds=bounds)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 24, CG iterations: 7, optimality: 4.52e-09, constraint violation: 0.00e+00, execution time: 0.04 s.


In [25]:
res.x

array([0.41494531, 0.17010937])